# SLAM tutorial

SLAM and factor graphs tutorial prepared for the ORIentate Seminars

Author: Matias Mattamala (matias@robots.ox.ac.uk, [mmattamala@github](https://github.com/mmattamala))

Date: 23/11/2023


> ⚠️ **Warning**: Before proceeding, make sure you installed all the required dependencies in the [`requirements.txt`](../requirements.txt) file!


## Preliminaries

In [ ]:
%pip install -e ../

### Part 1: Inspecting a perfect SLAM graph
In this first part we will check how a pose graph looks like. For that, we will use some data avaialble in the [assets](../assets/) folder.

In [ ]:
import slam_tutorial
from slam_tutorial.io import load_pose_graph
import numpy as np

graph = load_pose_graph(
    slam_tutorial.ASSETS_DIR + "/ground_truth.slam",
    clouds_path=slam_tutorial.ASSETS_DIR + "/individual_clouds",
)

The `graph` object stores the **nodes** and **edges** of the pose graph.

Each **node** is defined by:
- A _pose_ $\mathbf{T}_{\mathtt{WB}} \in $ SE(3) ($4\times4$ rigid body matrices), representing the pose of the _base_ in the _world_ frame.
- An _id_
- A _time stamp_ indicating when the node was created.

On the other side, each **edge** is defined by:
- A _parent id_, which indicates what is the origin node for the edge
- A _child id_ representing the node where the edge lands
- A _relative pose_ $\textbf{T}_{\text{parent, child}}$ indicating the rigid body transformation between from the child frame to the parent frame
- A _type_ indicating if the edge was produced by an odometry system (`odometry`) or a loop candidate (`loop`)

We can visualize the pose graph in 3D using the helper function `show_pose_graph`

In [ ]:
import slam_tutorial.visualization as vis

vis.show_pose_graph(
    graph,
    show_ids=True,
    show_frames=True,
    show_edges=False,
    show_nodes=False,
    show_clouds=True,
    odometry_color=vis.GRAY,
    loop_color=vis.RED,
)

### Part 2: A more realistic pose graph

In [ ]:
import slam_tutorial.pose_graph as pg

graph_with_drift = pg.add_odometry_drift(
    graph, noise_per_m=0.1, axis="xy", drift_type="random_walk"
)
vis.show_pose_graph(graph_with_drift)

### Part 3: Solving a pose graph with factor graphs

In [ ]:
initial_graph = pg.create_test_pose_graph(
    noise_per_m=0.1, axis="xy", drift_type="random_walk"
)
vis.show_pose_graph(initial_graph, window_name="Initial Graph", show_clouds=True)

In [ ]:
import gtsam
import numpy as np

# Create a factor graph container and add factors to it
factor_graph = gtsam.NonlinearFactorGraph()

# Add prior
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(0.001 * np.ones(6))
factor_graph.add(gtsam.PriorFactorPose3(0, initial_graph.get_node_pose(0), PRIOR_NOISE))

# Add odometry factors
for e in initial_graph.edges:
    if e["type"] == "odometry":
        ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Information(e["info"])
        factor_graph.add(
            gtsam.BetweenFactorPose3(
                e["parent_id"], e["child_id"], e["pose"], ODOMETRY_NOISE
            )
        )

# Add initial values
initial_estimate = gtsam.Values()
for i, node in enumerate(initial_graph.nodes):
    initial_estimate.insert(i, graph.get_node_pose(i))

#### Visualize factor graph

In [ ]:
factor_graph.saveGraph("test.dot", initial_estimate)
from graphviz import Source

s = Source.from_file("test.dot")
s.view()

#### Optimize

In [ ]:
# copy graph
import copy
optimized_graph = copy.deepcopy(initial_graph)

# Setup optimizer
parameters = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)

result = optimizer.optimize()
for n, _ in enumerate(optimized_graph.nodes):
        optimized_graph.set_node_pose(n, result.atPose3(n))
vis.show_pose_graph(optimized_graph, window_name="Optimized Graph", show_clouds=True)

In [ ]:
# Setup optimizer
parameters = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)

import open3d as o3d
import time
import copy

visualizer = o3d.visualization.Visualizer()
visualizer.create_window()

graph_vis = copy.deepcopy(pose_graph)
# Manually run optimizer
for i in range(10):
    print(f"iter: {i}")
    visualizer.clear_geometries()
    geometry = vis.to_geometries(graph_vis)
    for g in geometry:
        visualizer.add_geometry(g)

    optimizer.iterate()
    result = optimizer.values()
    for n, _ in enumerate(graph_vis.nodes):
        graph_vis.set_node_pose(n, result.atPose3(n))

    visualizer.poll_events()
    visualizer.update_renderer()
    time.sleep(1)

In [ ]:
# Compute marginals to show covariances
marginals = gtsam.Marginals(factor_graph, optimizer.values())

In [ ]:
import numpy as np
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering


def make_point_cloud(npts, center, radius):
    pts = np.random.uniform(-radius, radius, size=[npts, 3]) + center
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(pts)
    colors = np.random.uniform(0.0, 1.0, size=[npts, 3])
    cloud.colors = o3d.utility.Vector3dVector(colors)
    return cloud


app = gui.Application.instance
app.initialize()

points = make_point_cloud(100, (0, 0, 0), 1.0)

vis = o3d.visualization.O3DVisualizer("Open3D - 3D Text", 1024, 768)
vis.show_settings = True
vis.add_geometry("Points", points)
for idx in range(0, len(points.points)):
    vis.add_3d_label(points.points[idx], "{}".format(idx))
vis.reset_camera_to_default()

app.add_window(vis)
app.run()